In [3]:
import numpy as np

In [65]:
# load data
train_file = "C:/IITD/sem5/col774-ml/datasets/decision_tree/decision_tree/train.csv"
test_file = "C:/IITD/sem5/col774-ml/datasets/decision_tree/decision_tree/test.csv"
val_file = "C:/IITD/sem5/col774-ml/datasets/decision_tree/decision_tree/val.csv"
train_data = np.genfromtxt(train_file, delimiter=',')
test_data = np.genfromtxt(test_file, delimiter=',')
val_data = np.genfromtxt(val_file, delimiter=',')

In [5]:
def extract_data(data):
    x = data[:, :-1]
    y = data[:, -1].astype('int')
    return x, y

In [66]:
train_data = train_data[2: :]
x, y = extract_data(train_data)
num_attributes = x.shape[1]
num_labels = 7

In [120]:
j = 51
d = train_data[2:, :]
x = d[:, :-1]
f = d[:, j] == 1
e = np.where(d[:, j] == 1)
d2 = d[x[:, j] == 1]
d2[:, j]

array([1., 1., 1., ..., 1., 1., 1.])

In [7]:
def entropy(y):
    counts = np.bincount(y, minlength=num_labels+1) # count occurences of each label
    prob = counts / y.size # probability for each label
    # print("entr", counts, prob)
    summation = 0
    for k in range(1, num_labels + 1):
        if prob[k] != 0:
            summation += prob[k] * np.log2(prob[k])
    return -summation
    # return -sum(prob[k] * np.log2(prob[k]) for k in range(1, num_labels + 1))

In [57]:
yy = np.array([3,5, 3, 6, 3, 2])
entropy(yy)

entr [0 0 1 3 0 1 1 0] [0.         0.         0.16666667 0.5        0.         0.16666667
 0.16666667 0.        ]


1.7924812503605778

In [8]:
def choose_best_attribute(D):
    x, y = extract_data(D)
    max_MI = -1
    max_j = -1
    for j in range(num_attributes):
        if j > 10:
            # discrete attribute
            D0 = D[x[:, j] == 0]
            D1 = D[x[:, j] == 1]
        else:
            # continuous attribute 
            median = np.median(x[:, j])
            D0 = D[x[:, j] <= median]
            D1 = D[x[:, j] > median]
        x0, y0 = extract_data(D0)
        x1, y1 = extract_data(D1)
        p0 = y0.size / y.size
        p1 = y1.size / y.size
        MI = entropy(y) - (entropy(y0) * p0 + entropy(y1) * p1) # mutual information
        # print("MI", MI, j, entropy(y), entropy(y0), entropy(y1))
        if MI > max_MI:
            max_MI = MI
            max_j = j
    return max_j

In [10]:
choose_best_attribute(train_data)

9

In [39]:
class Node:
    def __init__(self, attribute):
        self.attribute = attribute
        self.children = None

    def create_leaf(class_label):
        leaf = Node(attribute=None)
        leaf.set_majority_class(class_label)
        return leaf

    def is_leaf(self):
        return self.children == None

    def add_child(self, node):
        if self.children == None:
            self.children = [node]
        else:
            self.children.append(node)

    def set_majority_class(self, majority_class):
        self.majority_class = majority_class

    def set_attribute(self, attribute):
        self.attribute = attribute

    def set_attribute_median(self, attribute_median):
        self.attribute_median = attribute_median

In [37]:
np.bincount(y, minlength=num_labels+1)[1:]

array([118729, 158746,  20062,   1546,   5357,   9754,  11535],
      dtype=int64)

In [10]:
num_nodes = 0

def grow_tree(D):
    print("gt", D.shape, num_nodes)
    x, y = extract_data(D)
    if np.all(y == y[0]):
        num_nodes += 1
        return Node(y[0])
    else:
        j = choose_best_attribute(D)
        if j > 10:
            # discrete attribute
            D0 = D[x[:, j] == 0]
            D1 = D[x[:, j] == 1]
        else:
            # continuous attribute 
            median = np.median(x[:, j])
            D0 = D[x[:, j] <= median]
            D1 = D[x[:, j] > median]
        node = Node(j)
        node.add_child(grow_tree(D0))
        node.add_child(grow_tree(D1))
        num_nodes += 1
        return node

In [11]:
dtree = grow_tree(train_data[:8])

UnboundLocalError: local variable 'num_nodes' referenced before assignment

In [56]:
class DecisionTree:

    def __init__(self, train_data):
        self.train_data = train_data
        self.num_nodes = 0

    def grow_tree(self, D):
        print("gt", D.shape, self.num_nodes)
        x, y = extract_data(D)
        if np.all(y == y[0]):
            # leaf node
            self.num_nodes += 1
            return Node.create_leaf(y[0])
        else:
            majority_class = np.argmax(np.bincount(y)[1:])
            j = choose_best_attribute(D)
            node = Node(j)
            self.num_nodes += 1
            if j > 10:
                # discrete attribute
                D0 = D[x[:, j] == 0]
                D1 = D[x[:, j] == 1]
            else:
                # continuous attribute 
                attribute_median = np.median(x[:, j])
                D0 = D[x[:, j] <= attribute_median]
                D1 = D[x[:, j] > attribute_median]
                node.set_attribute_median(attribute_median)
            node.add_child(self.grow_tree(D0))
            node.add_child(self.grow_tree(D1))
            return node

    def train(self):
        self.root = self.grow_tree(self.train_data)

    def predict(self, x):
        return self.predict_node(self.root, x)

    def predict_node(self, node, x):
        if node.is_leaf():
            return node.majority_class
        else:
            j = node.attribute
            if j > 10:
                # discrete attribute
                return self.predict_node(node.children[int(x[j])], x)
            else:
                # continuous attibute
                if x[j] <= node.attribute_median:
                    return self.predict_node(node.children[0], x)
                else:
                    return self.predict_node(node.children[1], x)



In [75]:
dtree = DecisionTree(train_data[:7])
dtree.train()
t = 3
print(dtree.predict(x[t]), y[t], dtree.num_nodes)
# dtree.num_nodes

gt (7, 55) 0
gt (4, 55) 1
gt (3, 55) 2
5 5 3
